# Import Libraries and files 

In [2]:
# Basic Libraries
import os
import pandas as pd

# Pre-processing
import fitz # install PyMuPDF
import requests
from tqdm import tqdm
from spacy.lang.en import English 
import re

In [3]:
# Get PDF document path
filename = "human-nutrition-text.pdf"

# Download PDF
if not os.path.exists(filename):
    print("File doesn't exist, downloading...")
    
    # The URL of the PDF you want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # Send a GET request to the url
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file and save it
        with open(filename,"wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been download and saved as {filename}")
    else: print(f"[INFO] Failed to download the file. Status Code: {response.status_code}")
        
else:
  print(f"File {filename} exists.")

File human-nutrition-text.pdf exists.


# Define Functions

In [4]:
# Format text read from PDF
def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n"," ").strip()
    return cleaned_text

# Provide pdf name
def open_and_read_pdf(filename: str)->list[dict]:
    doc = fitz.open(filename)
    pages_and_texts=[]
    for page_number,page in tqdm(enumerate(doc)):
        text = page.get_text()
        formatted_text = text_formatter(text)

        # store pages information and texts in a dictionary
        pages_and_texts.append({"page_number": page_number - 41,  # adjust page numbers since our PDF starts on page 42
                        "page_char_count": len(text),
                        "page_word_count": len(text.split(" ")),
                        "page_sentence_count_raw": len(text.split(". ")),
                        # "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                        "text": text})
        
    return pages_and_texts

def read_pages(pages,page_number:int):
    # insert pages_and_text and page number to retrieve the specified page in the pdf
    content  = pages[page_number+41]
    return content

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

In [5]:
def text_processor(pages_and_texts,num_sentence_chunk_size=10,min_token_length=30):
    nlp = English()
    # Add a sentencizer pipeline
    nlp.add_pipe("sentencizer")

    # Sentenciaing texts on each page to sentences
    for item in tqdm(pages_and_texts):
        item["sentences"] = list(nlp(item["text"]).sents)
        
        # Make sure all sentences are strings
        item["sentences"] = [str(sentence) for sentence in item["sentences"]]
        
        # Count the sentences 
        item["page_sentence_count_spacy"] = len(item["sentences"])

    # Loop through pages and texts and split sentences into chunks
    for item in tqdm(pages_and_texts):
        item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                            slice_size=num_sentence_chunk_size)
        item["num_chunks"] = len(item["sentence_chunks"])

    # Split each chunk into its own item
    pages_and_chunks = []
    for item in tqdm(pages_and_texts):
        for sentence_chunk in item["sentence_chunks"]:
            chunk_dict = {}
            chunk_dict["page_number"] = item["page_number"]
            
            # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
            joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
            chunk_dict["sentence_chunk"] = joined_sentence_chunk

            # Get stats about the chunk
            chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
            chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
            chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
            
            pages_and_chunks.append(chunk_dict)

    df = pd.DataFrame(pages_and_chunks)
    pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
    return pages_and_chunks, pages_and_chunks_over_min_token_len

# Embedding

In [6]:
# Pre-processing
pages_and_texts = open_and_read_pdf("human-nutrition-text.pdf")
pages_and_chunks,pages_and_chunks_over_min_token_len = text_processor(pages_and_texts)
read_pages(pages_and_chunks_over_min_token_len,258)

1208it [00:01, 820.84it/s]
100%|██████████| 1208/1208 [00:00<00:00, 30199.85it/s]


{'page_number': 196,
 'sentence_chunk': 'Potassium also is involved in protein synthesis, energy metabolism, \nand platelet function, and acts as a buffer in blood, playing a role in \nacid-base balance.\nImbalances of Potassium \nInsufficient potassium levels in the body (hypokalemia) can be \ncaused by a low dietary intake of potassium or by high sodium \nintakes, but more commonly it results from medications that \nincrease water excretion, mainly diuretics. The signs and symptoms \nof hypokalemia are related to the functions of potassium in nerve \ncells and consequently skeletal and smooth-muscle contraction.\nThe signs and symptoms include muscle weakness and cramps, \nrespiratory distress, and constipation. Severe potassium depletion \ncan cause the heart to have abnormal contractions and can even \nbe fatal. High levels of potassium in the blood, or hyperkalemia, \nalso affects the heart. It is a silent condition as it often displays \nno signs or symptoms. Extremely high level

### Embedding our text chunks

While humans understand text, machines understand numbers best.

An [embedding](https://vickiboykis.com/what_are_embeddings/index.html) is a broad concept.

But one of my favourite and simple definitions is "a useful numerical representation".

The most powerful thing about modern embeddings is that they are *learned* representations.

Meaning rather than directly mapping words/tokens/characters to numbers directly (e.g. `{"a": 0, "b": 1, "c": 3...}`), the numerical representation of tokens is learned by going through large corpuses of text and figuring out how different tokens relate to each other.

Ideally, embeddings of text will mean that similar meaning texts have similar numerical representation.

> **Note:** Most modern NLP models deal with "tokens" which can be considered as multiple different sizes and combinations of words and characters rather than always whole words or single characters. For example, the string `"hello world!"` gets mapped to the token values `{15339: b'hello', 1917: b' world', 0: b'!'}` using [Byte pair encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding) (or BPE via OpenAI's [`tiktoken`](https://github.com/openai/tiktoken) library). Google has a tokenization library called [SentencePiece](https://github.com/google/sentencepiece).

Our goal is to turn each of our chunks into a numerical representation (an embedding vector, where a vector is a sequence of numbers arranged in order).

Once our text samples are in embedding vectors, us humans will no longer be able to understand them.

However, we don't need to.

The embedding vectors are for our computers to understand.

We'll use our computers to find patterns in the embeddings and then we can use their text mappings to further our understanding.

Enough talking, how about we import a text embedding model and see what an embedding looks like.

To do so, we'll use the [`sentence-transformers`](https://www.sbert.net/docs/installation.html) library which contains many pre-trained embedding models.

Specifically, we'll get the `all-mpnet-base-v2` model (you can see the model's intended use on the [Hugging Face model card](https://huggingface.co/sentence-transformers/all-mpnet-base-v2#intended-uses)).

In [7]:
# Requires !pip install sentence-transformers
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cpu") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07981393e-02  3.03164497e-02 -2.01217979e-02  6.86483979e-02
 -2.55255587e-02 -8.47692136e-03 -2.07078308e-04 -6.32377341e-02
  2.81606130e-02 -3.33353430e-02  3.02634742e-02  5.30720465e-02
 -5.03526554e-02  2.62287818e-02  3.33313867e-02 -4.51579243e-02
  3.63044068e-02 -1.37113256e-03 -1.20170955e-02  1.14946328e-02
  5.04511520e-02  4.70857695e-02  2.11913157e-02  5.14607355e-02
 -2.03746576e-02 -3.58888917e-02 -6.67874468e-04 -2.94393655e-02
  4.95858677e-02 -1.05639929e-02 -1.52013954e-02 -1.31754915e-03
  4.48197387e-02  1.56023372e-02  8.60379657e-07 -1.21398771e-03
 -2.37978715e-02 -9.09400522e-04  7.34478515e-03 -2.53933389e-03
  5.23370095e-02 -4.68043461e-02  1.66214556e-02  4.71579246e-02
 -4.15599495e-02  9.01974272e-04  3.60279791e-02  3.42214853e-02
  9.68228132e-02  5.94828278e-02 -1.64984707e-02 -3.51249464e-02
  5.92516921e-03 -7.07960688e-04 -2.4103

In [8]:
# sentence holds the last sentence in a list of 4 sentences
print(sentence)
# embeddings is an array that holds 4 embedded vectors for 4 sentences
# while embeddings_dict is a dictionary that maps each vector to their respective sentence
print(embeddings)
# (4,768) represents 4 sentences
# 769 in length for each vector (same length)
print(embeddings.shape)

Learn to use embeddings well and you'll be well on your way to being an AI engineer.
[[-0.02079814  0.03031645 -0.0201218  ...  0.01136523 -0.06564108
  -0.02208491]
 [ 0.04317182 -0.05387013 -0.03780445 ... -0.00885381 -0.05962484
  -0.02378257]
 [-0.02986109 -0.01375232 -0.04754018 ...  0.05511304  0.02200251
  -0.02511706]
 [-0.0220731   0.0208951  -0.06030055 ...  0.02945551  0.00352996
  -0.06104216]]
(4, 768)


Our embedding has a shape of `(768,)` meaning it's a vector of 768 numbers which represent our text in high-dimensional space, too many for a human to comprehend but machines love high-dimensional space.

> **Note:** No matter the size of the text input to our `all-mpnet-base-v2` model, it will be turned into an embedding size of `(768,)`. This value is fixed. So whether a sentence is 1 token long or 1000 tokens long, it will be truncated/padded with zeros to size 384 and then turned into an embedding vector of size `(768,)`. Of course, other embedding models may have different input/output shapes.

Let's try embedding our `pages_and_chunks_over_min_token_len`

We will try to create embeddings on the CPU, and time it to see how long it takes.

In [39]:
# # Uncomment to see how long it takes to create embeddings on CPU
# # Make sure the model is on the CPU ( z-book: [03:41<00:00,  7.60it/s])
# embedding_model.to("cpu")

# # Embed each chunk one by one
# for item in tqdm(pages_and_chunks_over_min_token_len):
#     item["embedding"] = embedding_model.encode(item["sentence_chunk"])

100%|██████████| 1683/1683 [03:41<00:00,  7.60it/s]


In [9]:
# Send the model to the GPU
embedding_model.to("cuda") 

# Create embeddings one by one on the GPU (z-book: [00:39<00:00, 42.22it/s])
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

100%|██████████| 1683/1683 [01:42<00:00, 16.46it/s]


Woah! Looks like the embeddings get created much faster (~2x faster on my machine) on the GPU!

You'll likely notice this trend with many of your deep learning workflows. If you have access to a GPU, especially a NVIDIA GPU, you should use one if you can.

But what if I told you we could go faster again?

You see many modern models can handle batched predictions.

This means computing on multiple samples at once.

Those are the types of operations where a GPU flourishes!

We can perform batched operations by turning our target text samples into a single list and then passing that list to our embedding model.

In [10]:
pages_and_chunks_over_min_token_len[0]

{'page_number': -39,
 'sentence_chunk': 'Human Nutrition: 2020 \nEdition \nUNIVERSITY OF HAWAI‘I AT MĀNOA \nFOOD SCIENCE AND HUMAN \nNUTRITION PROGRAM \nALAN TITCHENAL, SKYLAR HARA, \nNOEMI ARCEO CAACBAY, WILLIAM \nMEINKE-LAU, YA-YUN YANG, MARIE \nKAINOA FIALKOWSKI REVILLA, \nJENNIFER DRAPER, GEMADY \nLANGFELDER, CHERYL GIBBY, CHYNA \nNICOLE CHUN, AND ALLISON \nCALABRESE',
 'chunk_char_count': 320,
 'chunk_word_count': 42,
 'chunk_token_count': 80.0,
 'embedding': array([ 6.74242675e-02,  9.02281553e-02, -5.09548699e-03, -3.17545645e-02,
         7.39082247e-02,  3.51976417e-02, -1.97986849e-02,  4.67692614e-02,
         5.35727181e-02,  5.01233246e-03,  3.33929323e-02, -1.62217882e-03,
         1.76080614e-02,  3.62653658e-02, -3.16694262e-04, -1.07118310e-02,
         1.54257976e-02,  2.62176320e-02,  2.77655781e-03,  3.64942364e-02,
        -4.44109775e-02,  1.89361889e-02,  4.90117520e-02,  1.64020006e-02,
        -4.85782698e-02,  3.18291178e-03,  2.72992607e-02, -2.04759696e-03,


In [11]:
len(pages_and_chunks_over_min_token_len)

1683

In [12]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks

['Human Nutrition: 2020 \nEdition \nUNIVERSITY OF HAWAI‘I AT MĀNOA \nFOOD SCIENCE AND HUMAN \nNUTRITION PROGRAM \nALAN TITCHENAL, SKYLAR HARA, \nNOEMI ARCEO CAACBAY, WILLIAM \nMEINKE-LAU, YA-YUN YANG, MARIE \nKAINOA FIALKOWSKI REVILLA, \nJENNIFER DRAPER, GEMADY \nLANGFELDER, CHERYL GIBBY, CHYNA \nNICOLE CHUN, AND ALLISON \nCALABRESE',
 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and \nHuman Nutrition Program is licensed under a Creative Commons Attribution 4.0 \nInternational License, except where otherwise noted.',
 'Contents \nPreface \nUniversity of Hawai‘i at Mānoa Food Science and \nHuman Nutrition Program and Human Nutrition \nProgram \nxxv \nAbout the Contributors \nUniversity of Hawai‘i at Mānoa Food Science and \nHuman Nutrition Program and Human Nutrition \nProgram \nxxvi \nAcknowledgements \nUniversity of Hawai‘i at Mānoa Food Science and \nHuman Nutrition Program and Human Nutrition \nProgram \nxl \nPart I. Chapter 1. Basic Concepts in Nutr

In [13]:
# Put all text chunks into a list
# 1 chunk 1 element in the list
# total 1683 elements in one list
len(text_chunks)

1683

In [15]:
# Define GPU device
embedding_model.to("cuda") 

# Embed all texts in batches (z-book: 34.4s)
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

Amost 2x improvement in my case.

So the tip here is to use a GPU when you can and use batched operations if you can too.

Now let's save our chunks and their embeddings so we could import them later if we wanted.

### Save embeddings to file

Since creating embeddings can be a timely process (not so much for this case but it can be for way larger datasets), let's turn our `pages_and_chunks_over_min_token_len` list of dictionaries into a DataFrame and save it.

In [17]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

Load the file to make sure we saved what we want

In [18]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 \nEdition \nUNIVERSITY O...,320,42,80.00,[ 6.74242675e-02 9.02281553e-02 -5.09548699e-...
1,-38,Human Nutrition: 2020 Edition by University of...,212,30,53.00,[ 5.52156307e-02 5.92139214e-02 -1.66167393e-...
2,-37,Contents \nPreface \nUniversity of Hawai‘i at ...,797,116,199.25,[ 2.79801916e-02 3.39813791e-02 -2.06426457e-...
3,-36,Lifestyles and Nutrition \nUniversity of Hawai...,976,144,244.00,[ 6.82567060e-02 3.81274708e-02 -8.46855436e-...
4,-35,The Cardiovascular System \nUniversity of Hawa...,1037,152,259.25,[ 3.30264568e-02 -8.49767309e-03 9.57160536e-...


### Chunking and embedding questions

> **Which embedding model should I use?**

This depends on many factors. My best advice is to experiment, experiment, experiment! 

If you want the model to run locally, you'll have to make sure it's feasible to run on your own hardware. 

A good place to see how different models perform on a wide range of embedding tasks is the [Hugging Face Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

> **What other forms of text chunking/splitting are there?**

There are a fair few options here too. We've kept it simple with groups of sentences.

For more, [Pinecone has a great guide on different kinds of chunking](https://www.pinecone.io/learn/chunking-strategies/) including for different kinds of data such as markdown and LaTeX.

Libraries such as [LangChain also have a good amount of in-built text splitting options](https://python.langchain.com/docs/modules/data_connection/document_transformers/).

> **What should I think about when creating my embeddings?**

Our model turns text inputs up to 384 tokens long in embedding vectors of size 768.

Generally, the larger the vector size, the more information that gets encoded into the embedding (however, this is not always the case, as smaller, better models can outperform larger ones).

Though with larger vector sizes comes larger storage and compute requirements.

Our model is also relatively small (420MB) in size compared to larger models that are available.

Larger models may result in better performance but will also require more compute.

So some things to think about:
* Size of input - If you need to embed longer sequences, choose a model with a larger input capacity.
* Size of embedding vector - Larger is generally a better representation but requires more compute/storage.
* Size of model - Larger models generally result in better embeddings but require more compute power/time to run.
* Open or closed - Open models allow you to run them on your own hardware whereas closed models can be easier to setup but require an API call to get embeddings.

> **Where should I store my embeddings?**

If you've got a relatively small dataset, for example, under 100,000 examples (this number is rough and only based on first hand experience), `np.array` or `torch.tensor` can work just fine as your dataset.

But if you've got a production system and want to work with 100,000+ embeddings, you may want to look into a [vector database]( https://en.wikipedia.org/wiki/Vector_database) (these have become very popular lately and there are many offerings).

### Document Ingestion and Embedding Creation Extensions

One major extension to the workflow above would to functionize it.

Or turn it into a script.

As in, take all the functionality we've created and package it into a single process (e.g. go from document -> embeddings file).

So you could input a document on one end and have embeddings come out the other end. The hardest part of this is knowing what kind of preprocessing your text may need before it's turned into embeddings. Cleaner text generally means better results.

